In [3]:
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
from os.path import join

In [4]:
region = 'borde_rural'

geojson_fp = join('..', '..', 'data', region, 'train-'+region+'.geojson')
image_fp = join('..', '..', 'data', region, region+'_ortho-cog.tif')
roof_dir = join('..', '..', 'data', region, 'roofs')

In [ ]:
with rasterio.open(image_fp) as src:
    profile = src.profile
    print(profile['crs'])
    print(src.profile)
    print(src.bounds)

## 1. Get an overview of the scenery

In [ ]:
with rasterio.open(image_fp) as src:
    oviews = src.overviews(1)
    oview = oviews[-1]
    print('Decimation factor = {}'.format(oview))
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview')

## 2. Cut out roofs and store as separate files

In [5]:
import rasterio
from rasterio.mask import mask
from os import makedirs
from os.path import exists

Create subfolders for each material

In [32]:
materials = ['healthy_metal', 'irregular_metal', 'concrete_cement', 'incomplete', 'other']
for mat in materials:
    directory = join(roof_dir, mat) 
    if not exists(directory):
        makedirs(directory)

Coordinates in GeoJSON label file need to be converted to CSR format of the image

In [33]:
from pyproj import Proj, transform
outProj = Proj(init=profile['crs']) # CRS format of image
inProj = Proj(init='epsg:4326') # lat/lon coordinate format

#### Cutting out roofs
1. Extract the roof id and polygon coordinates from GeoJSON label file
2. Cut the polygon out of the image
3. Save the cut out image of the roof to file with id as name

In [34]:
with open(geojson_fp) as geojson:
    geoms = json.loads(geojson.read())
    roofs = geoms['features']

for roof in roofs:
    roof_id = roof['id']
    roof_geom = roof['geometry']
    roof_material = roof['properties']['roof_material']
    coord = roof_geom['coordinates'][0]
    print(roof_id)
    for c in coord:
        c[0], c[1] = transform(inProj, outProj, c[0], c[1])

    print(roof_geom)
    with rasterio.open(image_fp) as image:
        roof_image, roof_transform = mask(image, [roof_geom], filled=True, crop=True)
    #show(roof_image)
    roof_meta = image.meta.copy()
    
    # save the resulting raster
    roof_meta.update({"driver": "GTiff",
        "dtype": rasterio.uint8,
        "height": roof_image.shape[1],
        "width": roof_image.shape[2],
        "transform": roof_transform,
        "tiled": True,
        "compress": 'lzw'})
    roof_image_fp = join(roof_dir, roof_material, str(roof_id)+".tif")

    with rasterio.open(roof_image_fp, "w", **roof_meta) as dest:
        dest.write(roof_image)                             

{'type': 'Polygon', 'coordinates': [[[593322.0146307958, 503544.01200439036], [593325.7023995072, 503555.3296394013], [593331.550536128, 503553.42406679504], [593327.8627674168, 503542.10643178405], [593322.0146307958, 503544.01200439036]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593314.707898139, 503546.9842346236], [593318.436816822, 503557.20868907863], [593324.7391751767, 503554.91018191347], [593321.0102564931, 503544.6857274584], [593314.707898139, 503546.9842346236]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.4

{'type': 'Polygon', 'coordinates': [[[593303.6858776889, 503524.56866494933], [593308.24292888, 503522.9972679875], [593304.4289352482, 503511.93668645807], [593299.8718840579, 503513.50808341795], [593303.6858776889, 503524.56866494933]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593290.9300562711, 503524.0732932406], [593295.8608245398, 503522.34086114913], [593293.9022865042, 503516.7665605863], [593288.971518235, 503518.4989926797], [593290.9300562711, 503524.0732932406]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.4

{'type': 'Polygon', 'coordinates': [[[593276.8119626626, 503515.1566025448], [593282.2354287509, 503512.9872161057], [593278.5457636323, 503503.76305331284], [593273.122297546, 503505.9324397482], [593276.8119626626, 503515.1566025448]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593282.6325802042, 503512.18437230785], [593286.3511655868, 503510.653190095], [593283.0041089848, 503502.52462405333], [593279.2855235981, 503504.0558062662], [593282.6325802042, 503512.18437230785]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.4

{'type': 'Polygon', 'coordinates': [[[593288.9485694501, 503461.9660656564], [593298.8296156286, 503458.09376377787], [593296.998359666, 503453.42090373865], [593287.1173134861, 503457.29320561886], [593288.9485694501, 503461.9660656564]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593296.8745167385, 503482.7716772927], [593301.7237497765, 503481.24506689043], [593299.4752181934, 503474.10267244454], [593294.6259851526, 503475.62928284693], [593296.8745167385, 503482.7716772927]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 50376

{'type': 'Polygon', 'coordinates': [[[593209.6890965595, 503429.9402848948], [593215.3858711731, 503445.91602239746], [593224.5502477266, 503442.5722633861], [593219.1011589641, 503426.3488400299], [593209.6890965595, 503429.9402848948]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593206.7168663272, 503421.89049468], [593209.4414107085, 503429.4449131881], [593218.6057872612, 503426.1011541764], [593215.2620282492, 503418.918264443], [593206.7168663272, 503421.89049468]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.4636400

...

{'type': 'Polygon', 'coordinates': [[[593302.0935322146, 503171.5571701359], [593305.5475415085, 503182.8863206227], [593310.1633400188, 503181.4790649805], [593306.709330726, 503170.1499144937], [593302.0935322146, 503171.5571701359]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593297.3310226871, 503177.24572318036], [593299.7737488976, 503185.34528903296], [593305.1362466312, 503183.72802781314], [593302.6935204233, 503175.6284619607], [593297.3310226871, 503177.24572318036]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.

{'type': 'Polygon', 'coordinates': [[[593318.8946074832, 503204.0480684509], [593323.921700869, 503202.4605652764], [593322.0696138323, 503196.50742837036], [593317.174812376, 503198.35951540613], [593318.8946074832, 503204.0480684509]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593295.3466437202, 503210.66266501317], [593297.8601904127, 503217.6741373706], [593303.0195757318, 503215.29288260645], [593298.918525863, 503204.97411196865], [593296.1403953063, 503206.0324474219], [593297.8601904133, 503210.3980811536], [593295.3466437202, 503210.66266501317]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618

{'type': 'Polygon', 'coordinates': [[[593344.426950213, 503267.3497575556], [593349.937118293, 503265.4647000545], [593347.8665404271, 503259.41224167857], [593342.3563723477, 503261.29729918024], [593344.426950213, 503267.3497575556]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593342.5748631753, 503260.4705771301], [593347.4696646312, 503259.27994974697], [593346.0144533888, 503253.85598056775], [593341.2519438637, 503255.311191812], [593342.5748631753, 503260.4705771301]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.463

{'type': 'Polygon', 'coordinates': [[[593240.8423680444, 503257.6659881864], [593244.7005407554, 503269.10271443625], [593250.3673870951, 503267.19100723777], [593246.5092143822, 503255.75428098807], [593240.8423680444, 503257.6659881864]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.46364000003), 'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'type': 'Polygon', 'coordinates': [[[593263.5965802207, 503249.5961803831], [593275.3730581761, 503246.2314723936], [593273.915350857, 503241.12949677935], [593262.1388729013, 503244.4942047689], [593263.5965802207, 503249.5961803831]]]}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 31315, 'height': 52318, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(0.03944, 0.0, 592371.56518,
       0.0, -0.03944, 503768.

ValueError: Input shapes do not overlap raster.